# Expliquer les Préférences : Pourquoi une cellule est-elle plus bénigne qu'une autre ?

**Question clé** : Comment expliquer à un patient ou à un médecin pourquoi le modèle considère qu'une cellule A est "plus bénigne" qu'une cellule B ?

Ce notebook propose une méthode pour **décomposer** cette comparaison en **arguments simples et compréhensibles** appelés *trade-offs* (compromis).

In [24]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from typing import List, Set, Optional, Dict
import warnings
warnings.filterwarnings('ignore')

## 1. Chargement des données

Le dataset Wisconsin Breast Cancer contient 683 échantillons de cellules avec 9 caractéristiques mesurées sur une échelle de 1 à 10 :

| Critère | Description | Interprétation |
|---------|-------------|----------------|
| ClumpThickness | Épaisseur des amas cellulaires | Plus c'est épais, plus c'est suspect |
| UniformityOfCellSize | Uniformité de la taille des cellules | Les cellules cancéreuses varient en taille |
| UniformityOfCellShape | Uniformité de la forme | Les cellules cancéreuses sont irrégulières |
| MarginalAdhesion | Adhésion marginale | Les cellules cancéreuses perdent leur adhérence |
| SingleEpithelialCellSize | Taille des cellules épithéliales | Cellules cancéreuses souvent agrandies |
| BareNuclei | Noyaux nus | Plus fréquents dans les tumeurs malignes |
| BlandChromatin | Chromatine fade | Texture anormale = signe de malignité |
| NormalNucleoli | Nucléoles normaux | Les nucléoles proéminents sont suspects |
| Mitoses | Activité mitotique | Division cellulaire rapide = danger |

In [25]:
# --- RATP: charger les données (Excel) ---
# If your file is in another folder, change the path accordingly.
df = pd.read_excel("content/RATP.xlsx", sheet_name=0)


# Some Excels create an unnamed index column; drop it if present
df = df.drop(columns=[c for c in df.columns if str(c).startswith("Unnamed")], errors="ignore")

station_col = "Metro station"

# The file contains a special row whose station name is "weights"
weights_mask = df[station_col].astype(str).str.strip().str.lower().eq("weights")
weights_row = df.loc[weights_mask].iloc[0]

df_stations = df.loc[~weights_mask].copy()

# Features = all columns except the station name
feature_names = [c for c in df_stations.columns if c != station_col]

# Make sure features are numeric
df_stations[feature_names] = df_stations[feature_names].apply(pd.to_numeric, errors="coerce")

# X = station data, weights = provided weights row
X = df_stations[feature_names].values
station_names = df_stations[station_col].astype(str).values
weights = weights_row[feature_names].astype(float).values

# French display names (optional but recommended for clean explanations)
noms_criteres = {
    "peak-entering-passengers/h": "Entrants pointe (/h)",
    "peak-passing-passengers/h": "Transit pointe (/h)",
    "off-peak-entering-passengers/h": "Entrants creuse (/h)",
    "off-peak-passing-passengers/h": "Transit creuse (/h)",
    "strategic priority [0,10]": "Priorité stratégique",
    "Station degradation level ([0,20] scale)": "Dégradation station",
    "connectivity index [0,100]": "Connectivité",
}

print(f"Dataset RATP : {len(X)} stations")
print("Colonnes utilisées :", feature_names)
print("Exemple station :", station_names[0])

KeyError: 'Metro station'

In [156]:
df.head()

,Benign,ClumpThickness,UniformityOfCellSize,UniformityOfCellShape,MarginalAdhesion,SingleEpithelialCellSize,BareNuclei,BlandChromatin,NormalNucleoli,Mitoses
0,0,5,1,1,1,2,1,3,1,1
1,0,5,4,4,5,7,10,3,2,1
2,0,3,1,1,1,2,2,3,1,1
3,0,6,8,8,1,3,4,3,7,1
4,0,4,1,1,3,2,1,3,1,1


## 2. Le modèle : Régression logistique

On utilise une **régression logistique** qui calcule un **score de malignité** pour chaque cellule :

$$\text{Score}(\text{cellule}) = \sum_{i=1}^{9} w_i \times \text{valeur}_i$$

- **Score élevé** → la cellule ressemble à une cellule maligne
- **Score bas** → la cellule ressemble à une cellule bénigne

Les poids $w_i$ indiquent l'importance de chaque critère pour prédire la malignité.

In [157]:
print("Poids (lus depuis la ligne 'weights' du fichier) :")
for name, w in zip(feature_names, weights):
    print(f"  {noms_criteres.get(name, name):<35} : {float(w):.3f}")


 Précision du modèle : 96.6%

 Importance des critères (poids du modèle) :
--------------------------------------------------
  Épaisseur des amas   : 0.525 █████
  Mitoses              : 0.483 ████
  Chromatine           : 0.433 ████
  Noyaux nus           : 0.381 ███
  Adhésion marginale   : 0.321 ███
  Uniformité forme     : 0.312 ███
  Nucléoles            : 0.211 ██
  Taille épithéliale   : 0.097 
  Uniformité taille    : 0.011 


In [158]:
EPS = 1e-6

def compute_omega(A, B, weights):
    # omega_i = w_i * (A_i - B_i)
    # sum(omega) = score(A) - score(B) > 0  <=>  A is MORE PRIORITY than B
    return weights * (A - B)


def classify_omega(omega, eps=EPS):
    pros = [i for i,v in enumerate(omega) if v > eps]     # arguments FOR "A more benign"
    cons = [i for i,v in enumerate(omega) if v < -eps]    # arguments AGAINST
    neutral = [i for i,v in enumerate(omega) if abs(v) <= eps]
    return pros, cons, neutral

In [159]:
def crit_name(i: int, feature_names: List[str]) -> str:
    # Human-friendly French name
    return noms_criteres[feature_names[i]]

def fmt_q1_11(exp11, feature_names):
    # exp11 = [(p,c), ...]
    lines = []
    for p, c in exp11:
        lines.append(f"- L'avantage sur {crit_name(p, feature_names)} compense l'inconvénient sur {crit_name(c, feature_names)}.")
    return "\n".join(lines) if lines else "- Dominance (aucun inconvénient)."

def fmt_q2_1m(exp1m, feature_names):
    # exp1m = [(p, [c1,c2,...]), ...]
    lines = []
    for p, cons_list in exp1m:
        cons_names = ", ".join(crit_name(c, feature_names) for c in cons_list)
        lines.append(f"- L'avantage sur {crit_name(p, feature_names)} compense : {cons_names}.")
    return "\n".join(lines) if lines else "- Dominance (aucun inconvénient)."

def fmt_q3_m1(expm1, feature_names):
    # expm1 = [([p1,p2,...], c), ...]
    lines = []
    for pros_list, c in expm1:
        pros_names = ", ".join(crit_name(p, feature_names) for p in pros_list)
        lines.append(f"- {pros_names} compensent l'inconvénient sur {crit_name(c, feature_names)}.")
    return "\n".join(lines) if lines else "- Dominance (aucun inconvénient)."

def fmt_q4(trade1m, tradem1, feature_names):
    lines = []
    if trade1m:
        lines.append("Trade-offs 1-m :")
        for p, cons_list in trade1m:
            cons_names = ", ".join(crit_name(c, feature_names) for c in cons_list)
            lines.append(f"- {crit_name(p, feature_names)} compense : {cons_names}.")
    if tradem1:
        lines.append("Trade-offs m-1 :")
        for pros_list, c in tradem1:
            pros_names = ", ".join(crit_name(p, feature_names) for p in pros_list)
            lines.append(f"- {pros_names} compensent : {crit_name(c, feature_names)}.")
    return "\n".join(lines) if lines else "- Dominance (aucun inconvénient)."


## 3. Comparer deux cellules : le problème

Quand on compare deux cellules A et B, on veut expliquer **pourquoi A est préférable que B** (ou inversement).

### Les trois types de critères

Pour chaque critère, on regarde la contribution au score :

- **Avantages (pros)** : critères où A est meilleure (plus bénigne) que B
- **Inconvénients (cons)** : critères où A est moins bien que B  
- **Neutres** : critères où A et B sont égales

### L'idée des trade-offs

Pour expliquer que A est globalement plus bénigne, on doit montrer comment ses **avantages compensent ses inconvénients**. C'est ce qu'on appelle un **trade-off** (compromis).

In [160]:
class ComparaisonCellules:
    """
    Compare deux stations (A vs B) et identifie les avantages/inconvénients.

    Convention RATP (priorité):
    - score(station) = sum_i w_i * valeur_i
    - Plus le score est élevé => station plus prioritaire
    - omega_i = w_i * (A_i - B_i)
      omega_i > 0 : argument POUR "A plus prioritaire que B"
      omega_i < 0 : argument CONTRE
    """

    def __init__(self, cellule_A: np.ndarray, cellule_B: np.ndarray,
                 weights: np.ndarray, feature_names: List[str]):
        self.A = cellule_A
        self.B = cellule_B
        self.weights = weights
        self.feature_names = feature_names

        # Scores de priorité (plus haut = plus prioritaire)
        self.score_A = float(np.dot(weights, cellule_A))
        self.score_B = float(np.dot(weights, cellule_B))

        # Contributions
        self.omega = compute_omega(cellule_A, cellule_B, weights)

        # Classification
        pros, cons, neutral = classify_omega(self.omega, eps=EPS)
        self.avantages = set(pros)
        self.inconvenients = set(cons)
        self.neutres = set(neutral)

        # A plus prioritaire ?
        self.difference = self.score_A - self.score_B
        self.A_plus_prioritaire = self.difference >= 0

    def afficher_comparaison(self, label_A="A", label_B="B"):
        print("\n" + "=" * 70)
        print(" COMPARAISON DE DEUX STATIONS (RATP)")
        print("=" * 70)

        print("\n Scores de priorité (plus haut = plus prioritaire) :")
        print(f"   Station {label_A} : {self.score_A:.2f}")
        print(f"   Station {label_B} : {self.score_B:.2f}")
        print(f"   → {'A est plus prioritaire' if self.A_plus_prioritaire else 'B est plus prioritaire'} "
              f"(différence : {abs(self.difference):.2f})")

        print("\n Détail critère par critère :")
        print(f"{'Critère':<30} {'A':>10} {'B':>10} {'Omega':>10} {'Verdict':<15}")
        print("-" * 80)

        for i, name in enumerate(self.feature_names):
            diff = self.omega[i]
            if diff > EPS:
                verdict = "Avantage A"
            elif diff < -EPS:
                verdict = "Inconv. A"
            else:
                verdict = "Égalité"

            print(f"{noms_criteres.get(name, name):<30} {self.A[i]:>10.2f} {self.B[i]:>10.2f} "
                  f"{diff:>+10.2f} {verdict}")

        print("\n Résumé :")
        print(f"    {len(self.avantages)} avantage(s) pour A")
        print(f"    {len(self.inconvenients)} inconvénient(s) pour A")
        print(f"    {len(self.neutres)} critère(s) neutre(s)")


## 4. Les types d'explications (trade-offs)

Pour expliquer pourquoi A est préférable malgré certains inconvénients, on utilise trois types d'arguments :

###  Type 1 : Échange simple (1-1)
"L'avantage sur le critère X compense l'inconvénient sur le critère Y"

*Exemple : "La cellule A a des amas plus épais que B, mais sa chromatine est bien meilleure, ce qui compense."*

###  Type 2 : Argument fort (1-m)
"Un seul avantage majeur compense plusieurs petits inconvénients"

*Exemple : "Bien que A soit légèrement moins bien sur l'uniformité et l'adhésion, son excellente chromatine compense tout."*

###  Type 3 : Accumulation (m-1)
"Plusieurs petits avantages s'additionnent pour compenser un gros inconvénient"

*Exemple : "L'inconvénient majeur sur les mitoses est compensé par les avantages combinés sur la chromatine et les noyaux nus."*

In [161]:
class TradeOff:
    """
    Un trade-off est un compromis entre des avantages et des inconvénients.
    """

    def __init__(self, avantages: Set[int], inconvenients: Set[int],
                 comparaison: ComparaisonCellules):
        self.avantages = avantages
        self.inconvenients = inconvenients
        self.comparaison = comparaison

        # Force du trade-off = somme des contributions
        self.force = sum(comparaison.omega[i] for i in avantages | inconvenients)
        self.est_valide = self.force >= 0  # Les avantages compensent-ils ?

    def get_type(self) -> str:
        """Retourne le type de trade-off."""
        n_av, n_inc = len(self.avantages), len(self.inconvenients)
        if n_av == 1 and n_inc == 1:
            return "échange"
        elif n_av == 1:
            return "argument_fort"
        else:
            return "accumulation"

    def expliquer(self) -> str:
        """Génère une explication en français."""
        comp = self.comparaison
        av_noms = [noms_criteres[comp.feature_names[i]] for i in self.avantages]
        inc_noms = [noms_criteres[comp.feature_names[i]] for i in self.inconvenients]

        type_to = self.get_type()

        if type_to == "échange":
            av_idx = list(self.avantages)[0]
            inc_idx = list(self.inconvenients)[0]
            return (f" Échange : La meilleure {av_noms[0].lower()} de A "
                   f"({comp.A[av_idx]:.0f} vs {comp.B[av_idx]:.0f}) "
                   f"compense sa moins bonne {inc_noms[0].lower()} "
                   f"({comp.A[inc_idx]:.0f} vs {comp.B[inc_idx]:.0f})")

        elif type_to == "argument_fort":
            av_idx = list(self.avantages)[0]
            inc_str = ", ".join(inc_noms)
            return (f" Argument fort : L'excellent score de A sur {av_noms[0].lower()} "
                   f"({comp.A[av_idx]:.0f} vs {comp.B[av_idx]:.0f}) "
                   f"compense à lui seul les faiblesses sur : {inc_str}")

        else:  # accumulation
            inc_idx = list(self.inconvenients)[0]
            av_str = ", ".join(av_noms)
            return (f" Accumulation : Les avantages combinés de A sur {av_str} "
                   f"compensent ensemble le désavantage sur {inc_noms[0].lower()} "
                   f"({comp.A[inc_idx]:.0f} vs {comp.B[inc_idx]:.0f})")

In [162]:
class ExplicationComplete:
    """
    Une explication complète est un ensemble de trade-offs qui couvre
    tous les inconvénients de la cellule A.
    """

    def __init__(self, trade_offs: List[TradeOff]):
        self.trade_offs = trade_offs
        self.longueur = len(trade_offs)

    def est_valide(self) -> bool:
        return all(to.est_valide for to in self.trade_offs)

    def afficher(self):
        """Affiche l'explication complète."""
        print("\n" + "─" * 70)
        print(" EXPLICATION DE LA PRÉFÉRENCE")
        print("─" * 70)

        if self.longueur == 0:
            print("\n Dominance totale : A est meilleure que B sur TOUS les critères !")
            print("   → Aucun compromis nécessaire.")
            return

        print(f"\nPourquoi A est préférable malgré certains inconvénients ?")
        print(f"Voici {self.longueur} argument(s) qui l'expliquent :\n")

        for i, to in enumerate(self.trade_offs, 1):
            print(f"{i}. {to.expliquer()}")
            print()

## 5. Algorithmes pour trouver les explications

In [163]:
def solve_explanation_11(omega, feature_names=None, eps=1e-6, verbose=False):
    n = len(omega)
    if feature_names is None:
        feature_names = [f"C{i}" for i in range(n)]

    pros, cons, neutral = classify_omega(omega, eps=eps)
    if sum(omega) <= eps:
        return None, "INVALID"  # A is not more benign than B
    if len(cons) == 0:
        return [], "TRIVIAL"
    if len(cons) > len(pros):
        return None, "INFEASIBLE"  # simple certificate for 1–1

    valid_pairs = [(p,c) for p in pros for c in cons if omega[p] + omega[c] > eps]
    for c in cons:
        if not any((p,c) in valid_pairs for p in pros):
            return None, "INFEASIBLE"

    m = gp.Model("explain_11")
    m.Params.OutputFlag = 1 if verbose else 0

    z = {(p,c): m.addVar(vtype=GRB.BINARY, name=f"z_{p}_{c}") for (p,c) in valid_pairs}
    m.update()

    # each con covered exactly once
    for c in cons:
        m.addConstr(gp.quicksum(z[p,c] for p in pros if (p,c) in z) == 1)

    # each pro used at most once
    for p in pros:
        m.addConstr(gp.quicksum(z[p,c] for c in cons if (p,c) in z) <= 1)

    # maximize total margin of chosen trade-offs
    m.setObjective(gp.quicksum(z[p,c] * (omega[p] + omega[c]) for (p,c) in valid_pairs), GRB.MAXIMIZE)

    m.optimize()
    if m.status == GRB.OPTIMAL:
        explanation = [(p,c) for (p,c) in valid_pairs if z[p,c].X > 0.5]
        return explanation, "OPTIMAL"
    if m.status == GRB.INFEASIBLE:
        return None, "INFEASIBLE"
    return None, f"STATUS_{m.status}"

def solve_explanation_1m(omega, eps=1e-6, verbose=False):
    pros, cons, neutral = classify_omega(omega, eps=eps)
    if sum(omega) <= eps:
        return None, "INVALID"
    if len(cons) == 0:
        return [], "TRIVIAL"
    if len(pros) == 0:
        return None, "INFEASIBLE"

    m = gp.Model("explain_1m")
    m.Params.OutputFlag = 1 if verbose else 0

    z = {(p,c): m.addVar(vtype=GRB.BINARY, name=f"z_{p}_{c}") for p in pros for c in cons}
    y = {p: m.addVar(vtype=GRB.BINARY, name=f"y_{p}") for p in pros}
    m.update()

    # each con assigned to exactly one pro
    for c in cons:
        m.addConstr(gp.quicksum(z[p,c] for p in pros) == 1)

    # link z and y
    for p in pros:
        for c in cons:
            m.addConstr(z[p,c] <= y[p])

    # validity per used pro: omega[p] + sum(assigned cons) >= eps
    M = 10_000
    for p in pros:
        m.addConstr(omega[p] + gp.quicksum(z[p,c] * omega[c] for c in cons) >= eps - M*(1 - y[p]))

    # minimize number of trade-offs (number of used pros)
    m.setObjective(gp.quicksum(y[p] for p in pros), GRB.MINIMIZE)

    m.optimize()
    if m.status == GRB.OPTIMAL:
        explanation = []
        for p in pros:
            if y[p].X > 0.5:
                cons_for_p = [c for c in cons if z[p,c].X > 0.5]
                explanation.append((p, cons_for_p))
        return explanation, "OPTIMAL"
    if m.status == GRB.INFEASIBLE:
        return None, "INFEASIBLE"
    return None, f"STATUS_{m.status}"

def solve_explanation_m1(omega, eps=1e-6, verbose=False):
    pros, cons, neutral = classify_omega(omega, eps=eps)
    if sum(omega) <= eps:
        return None, "INVALID"
    if len(cons) == 0:
        return [], "TRIVIAL"
    if len(pros) == 0:
        return None, "INFEASIBLE"

    m = gp.Model("explain_m1")
    m.Params.OutputFlag = 1 if verbose else 0

    z = {(p,c): m.addVar(vtype=GRB.BINARY, name=f"z_{p}_{c}") for p in pros for c in cons}
    m.update()

    # each pro used at most once (disjointness on pros)
    for p in pros:
        m.addConstr(gp.quicksum(z[p,c] for c in cons) <= 1)

    # validity for each con: sum(selected pros) + omega[c] >= eps
    for c in cons:
        m.addConstr(gp.quicksum(z[p,c] * omega[p] for p in pros) + omega[c] >= eps)

    # minimize number of links
    m.setObjective(gp.quicksum(z[p,c] for p in pros for c in cons), GRB.MINIMIZE)

    m.optimize()
    if m.status == GRB.OPTIMAL:
        explanation = []
        for c in cons:
            pros_for_c = [p for p in pros if z[p,c].X > 0.5]
            explanation.append((pros_for_c, c))
        return explanation, "OPTIMAL"
    if m.status == GRB.INFEASIBLE:
        return None, "INFEASIBLE"
    return None, f"STATUS_{m.status}"

def solve_explanation_combined_q4(omega, feature_names=None, eps=1e-6, verbose=False):
    """
    Q4 (combined): explanation mixing 1-m and m-1 trade-offs.

    Inputs
    - omega: np.array, omega_i = w_i * (B_i - A_i) so sum(omega) = score(B)-score(A)
    - A is "more benign" than B iff sum(omega) > 0

    Output
    - tradeoffs_1m: list of (p, [c1,c2,...]) for cons explained in 1-m mode
    - tradeoffs_m1: list of ([p1,p2,...], c) for cons explained in m-1 mode
    - status: "OPTIMAL", "INFEASIBLE", "TRIVIAL", "INVALID", etc.
    """
    n = len(omega)
    if feature_names is None:
        feature_names = [f"C{i}" for i in range(n)]

    # pros/cons/neutral based on omega sign
    pros = [i for i,v in enumerate(omega) if v > eps]
    cons = [i for i,v in enumerate(omega) if v < -eps]
    neutral = [i for i,v in enumerate(omega) if abs(v) <= eps]

    # validity of the comparison (A more benign than B)
    if float(sum(omega)) <= eps:
        return None, None, "INVALID"
    if len(cons) == 0:
        return [], [], "TRIVIAL"
    if len(pros) == 0:
        return None, None, "INFEASIBLE"

    model = gp.Model("explanation_combined_q4")
    model.Params.OutputFlag = 1 if verbose else 0

    M = 10_000.0

    # z[p,c] = 1 if pro p contributes to covering con c (in either mode)
    z = {(p,c): model.addVar(vtype=GRB.BINARY, name=f"z_{p}_{c}")
         for p in pros for c in cons}

    # mode1m[c] = 1 if con c is explained in 1-m mode (exactly one pro covers it)
    # mode1m[c] = 0 if con c is explained in m-1 mode (at least two pros cover it)
    mode1m = {c: model.addVar(vtype=GRB.BINARY, name=f"mode1m_{c}") for c in cons}

    # y[p] = 1 if pro p is used at all
    y = {p: model.addVar(vtype=GRB.BINARY, name=f"y_{p}") for p in pros}

    # w[p,c] = z[p,c] AND mode1m[c]  (used to sum only cons in 1-m mode for a pro)
    w = {(p,c): model.addVar(vtype=GRB.BINARY, name=f"w_{p}_{c}")
         for p in pros for c in cons}

    # has1m[p] = 1 if pro p covers at least one con in 1-m mode
    has1m = {p: model.addVar(vtype=GRB.BINARY, name=f"has1m_{p}") for p in pros}

    model.update()

    # Link z and y: if z[p,c]=1 then y[p]=1
    for p in pros:
        for c in cons:
            model.addConstr(z[p,c] <= y[p], name=f"link_{p}_{c}")

    # Each con must be covered by at least one pro
    for c in cons:
        model.addConstr(gp.quicksum(z[p,c] for p in pros) >= 1, name=f"cover_{c}")

    # Mode constraint:
    # if mode1m[c]=1 => exactly 1 pro covers c
    # if mode1m[c]=0 => at least 2 pros cover c
    for c in cons:
        nprosc = gp.quicksum(z[p,c] for p in pros)
        model.addConstr(nprosc <= 1 + M*(1 - mode1m[c]), name=f"mode1m_upper_{c}")
        model.addConstr(nprosc >= 2 - M*(mode1m[c]),   name=f"modem1_lower_{c}")

    # Validity per con (always): sum_p z[p,c]*omega[p] + omega[c] >= eps
    for c in cons:
        model.addConstr(gp.quicksum(z[p,c]*float(omega[p]) for p in pros) + float(omega[c]) >= eps,
                        name=f"valid_con_{c}")

    # Linearize w[p,c] = z[p,c] AND mode1m[c]
    for p in pros:
        for c in cons:
            model.addConstr(w[p,c] <= z[p,c],                 name=f"and1_{p}_{c}")
            model.addConstr(w[p,c] <= mode1m[c],              name=f"and2_{p}_{c}")
            model.addConstr(w[p,c] >= z[p,c] + mode1m[c] - 1, name=f"and3_{p}_{c}")

    # Define has1m[p] based on whether p covers any con in 1-m mode
    for p in pros:
        sumw = gp.quicksum(w[p,c] for c in cons)
        model.addConstr(has1m[p] <= sumw,            name=f"has1m_lb_{p}")
        model.addConstr(sumw <= M * has1m[p],        name=f"has1m_ub_{p}")

    # Validity of each 1-m trade-off for each pro that is used in 1-m mode:
    # omega[p] + sum_{c in cons} w[p,c]*omega[c] >= eps  if has1m[p]=1
    for p in pros:
        model.addConstr(float(omega[p]) + gp.quicksum(w[p,c]*float(omega[c]) for c in cons)
                        >= eps - M*(1 - has1m[p]),
                        name=f"valid_pro_1m_{p}")

    # Exclusivity: if p is used in m-1 mode for some c (i.e., z[p,c]=1 and mode1m[c]=0),
    # then p cannot cover any other con (so it remains "disjoint" like in Q3).
    for p in pros:
        for c in cons:
            model.addConstr(gp.quicksum(z[p,c2] for c2 in cons)
                            <= 1 + M*(1 - z[p,c]) + M*mode1m[c],
                            name=f"excl_{p}_{c}")

    # Objective (same spirit as the internat notebook Q4): minimize number of links
    model.setObjective(gp.quicksum(z[p,c] for p in pros for c in cons), GRB.MINIMIZE)

    model.optimize()

    if model.status == GRB.INFEASIBLE:
        return None, None, "INFEASIBLE"
    if model.status != GRB.OPTIMAL:
        return None, None, f"STATUS_{model.status}"

    # Build explanation
    cons_in_1m = [c for c in cons if mode1m[c].X > 0.5]
    cons_in_m1 = [c for c in cons if mode1m[c].X <= 0.5]

    # 1-m trade-offs: group cons by pro
    proto_cons = {}
    for c in cons_in_1m:
        for p in pros:
            if z[p,c].X > 0.5:
                proto_cons.setdefault(p, []).append(c)

    tradeoffs_1m = [(p, proto_cons[p]) for p in proto_cons]

    # m-1 trade-offs: one per con in m-1 mode
    tradeoffs_m1 = []
    for c in cons_in_m1:
        pros_for_c = [p for p in pros if z[p,c].X > 0.5]
        tradeoffs_m1.append((pros_for_c, c))

    return tradeoffs_1m, tradeoffs_m1, "OPTIMAL"

## 6. Exemples concrets

Comparons des cellules réelles du dataset pour illustrer les explications.

In [164]:
def analyser_et_expliquer(idx_A: int, idx_B: int, label_A: str = None, label_B: str = None):
    comp = ComparaisonCellules(X[idx_A], X[idx_B], weights, feature_names)

    if label_A is None:
        label_A = station_names[idx_A]
    if label_B is None:
        label_B = station_names[idx_B]

    print(f"\n{'='*70}")
    print(f"Comparaison : {label_A} (#{idx_A}) vs {label_B} (#{idx_B})")

    comp.afficher_comparaison(label_A="A", label_B="B")

    if comp.A_plus_prioritaire:
        omega = comp.omega

        exp11, st11 = solve_explanation_11(omega, feature_names=feature_names, verbose=False)
        if st11 in ["OPTIMAL", "TRIVIAL"]:
            print("Explication (solveur) de type 1-1 :", st11)
            print(fmt_q1_11(exp11, feature_names))
            return comp

        exp1m, st1m = solve_explanation_1m(omega, verbose=False)
        if st1m in ["OPTIMAL", "TRIVIAL"]:
            print("Explication (solveur) de type 1-m :", st1m)
            print(fmt_q2_1m(exp1m, feature_names))
            return comp

        expm1, stm1 = solve_explanation_m1(omega, verbose=False)
        if stm1 in ["OPTIMAL", "TRIVIAL"]:
            print("Explication (solveur) de type m-1 :", stm1)
            print(fmt_q3_m1(expm1, feature_names))
            return comp

        trade1m, tradem1, st4 = solve_explanation_combined_q4(omega, feature_names=feature_names, verbose=False)
        if st4 in ["OPTIMAL", "TRIVIAL"]:
            print("Explication (solveur) combinée (Q4) :", st4)
            print(fmt_q4(trade1m, tradem1, feature_names))
            return comp

        print("Aucune explication trouvée (même avec Q4).")
    else:
        print("Attention : B est en fait plus prioritaire que A (selon le score).")



### Exemple 1 : Cellule bénigne vs cellule maligne

Comparons une cellule clairement bénigne avec une cellule maligne.

In [165]:
scores_priorite = X @ weights

idx_plus_prioritaire = int(np.argmax(scores_priorite))
idx_moins_prioritaire = int(np.argmin(scores_priorite))

print("Exemple 1 : Station la plus prioritaire vs la moins prioritaire")
comp1 = analyser_et_expliquer(idx_plus_prioritaire, idx_moins_prioritaire, "Plus prioritaire", "Moins prioritaire")


Exemple 1 : Cas extrêmes

 Comparaison : Cellule Très bénigne (#481) vs Cellule Très maligne (#597)
   Diagnostic réel : Très bénigne=Bénigne, Très maligne=Maligne

 COMPARAISON DE DEUX CELLULES

 Scores de malignité :
   Cellule A : 2.77
   Cellule B : 26.30
   → A est plus bénigne (différence : 23.53)

 Détail critère par critère :
Critère                    A     B     Diff Verdict        
------------------------------------------------------------
Épaisseur des amas         1     8    +3.68  Avantage A
Uniformité taille          1    10    +0.10  Avantage A
Uniformité forme           1    10    +2.81  Avantage A
Adhésion marginale         1    10    +2.89  Avantage A
Taille épithéliale         1     6    +0.49  Avantage A
Noyaux nus                 1    10    +3.43  Avantage A
Chromatine                 1    10    +3.90  Avantage A
Nucléoles                  1    10    +1.90  Avantage A
Mitoses                    1    10    +4.35  Avantage A

 Résumé :
    9 avantage(s) pour A
   

### Exemple 2 : Deux cellules bénignes mais différentes

Comparons deux cellules bénignes pour voir les nuances.

In [166]:
ordre = np.argsort(-scores_priorite)
i, j = int(ordre[0]), int(ordre[1])

print("\nExemple 2 : Deux stations très prioritaires")
comp2 = analyser_et_expliquer(i, j, station_names[i], station_names[j])



 Exemple 2 : Deux cellules bénignes, profils différents

 Comparaison : Cellule Bénigne claire (#481) vs Cellule Bénigne ambiguë (#190)
   Diagnostic réel : Bénigne claire=Bénigne, Bénigne ambiguë=Bénigne

 COMPARAISON DE DEUX CELLULES

 Scores de malignité :
   Cellule A : 2.77
   Cellule B : 15.84
   → A est plus bénigne (différence : 13.07)

 Détail critère par critère :
Critère                    A     B     Diff Verdict        
------------------------------------------------------------
Épaisseur des amas         1     8    +3.68  Avantage A
Uniformité taille          1     4    +0.03  Avantage A
Uniformité forme           1     4    +0.94  Avantage A
Adhésion marginale         1     5    +1.28  Avantage A
Taille épithéliale         1     4    +0.29  Avantage A
Noyaux nus                 1     7    +2.28  Avantage A
Chromatine                 1     7    +2.60  Avantage A
Nucléoles                  1     8    +1.48  Avantage A
Mitoses                    1     2    +0.48  Avantage

### Exemple 3 : Cas intéressant avec trade-offs

Cherchons un cas où la cellule bénigne a vraiment des inconvénients à compenser.

In [167]:
np.random.seed(42)
trouve = False
n = len(X)

for _ in range(2000):
    i, j = np.random.choice(n, 2, replace=False)
    if scores_priorite[i] > scores_priorite[j]:
        comp_test = ComparaisonCellules(X[i], X[j], weights, feature_names)
        if comp_test.A_plus_prioritaire and len(comp_test.inconvenients) >= 2:
            print("\nExemple 3 : Cas avec plusieurs compromis")
            comp3 = analyser_et_expliquer(i, j, station_names[i], station_names[j])
            trouve = True
            break

if not trouve:
    print("Pas de cas avec plusieurs compromis trouvé dans l'échantillon.")



 Exemple 3 : Cas avec plusieurs compromis

 Comparaison : Cellule Bénigne (#285) vs Cellule Maligne (#279)
   Diagnostic réel : Bénigne=Bénigne, Maligne=Maligne

 COMPARAISON DE DEUX CELLULES

 Scores de malignité :
   Cellule A : 10.86
   Cellule B : 11.47
   → A est plus bénigne (différence : 0.62)

 Détail critère par critère :
Critère                    A     B     Diff Verdict        
------------------------------------------------------------
Épaisseur des amas         5     6    +0.53  Avantage A
Uniformité taille          3     1    -0.02  Inconvénient A
Uniformité forme           4     3    -0.31  Inconvénient A
Adhésion marginale         3     1    -0.64  Inconvénient A
Taille épithéliale         4     4    +0.00  Égalité
Noyaux nus                 5     5    +0.00  Égalité
Chromatine                 4     5    +0.43  Avantage A
Nucléoles                  7    10    +0.63  Avantage A
Mitoses                    1     1    +0.00  Égalité

 Résumé :
    3 avantage(s) pour A
  

## 7. Statistiques d'explicabilité

Quelle proportion des comparaisons peut-on expliquer avec chaque type de trade-off ?

In [168]:
def calculer_statistiques(n_echantillons: int = 500):
    """
    Stats based on MILP solvers (Q1-Q4).
    """
    np.random.seed(42)
    n = len(X)

    # Build pairs where A is more benign than B according to the model score
    paires = []
    tentatives = 0
    while len(paires) < n_echantillons and tentatives < n_echantillons * 20:
        i, j = np.random.choice(n, 2, replace=False)
        if scores_priorite[i] > scores_priorite[j]:
            paires.append((i, j))
        tentatives += 1

    stats = {
        "total": len(paires),
        "dominance": 0,
        "q1_11": 0,
        "q2_1m": 0,
        "q3_m1": 0,
        "q4_combined": 0,
        "explicable": 0
    }

    for i, j in paires:
        comp = ComparaisonCellules(X[i], X[j], weights, feature_names)

        # dominance = no cons
        if len(comp.inconvenients) == 0:
            stats["dominance"] += 1
            stats["explicable"] += 1
            continue

        omega = comp.omega

        # Q1
        exp11, st11 = solve_explanation_11(omega, feature_names=feature_names, verbose=False)
        if st11 == "OPTIMAL":
            stats["q1_11"] += 1
            stats["explicable"] += 1
            continue

        # Q2
        exp1m, st1m = solve_explanation_1m(omega, verbose=False)
        if st1m == "OPTIMAL":
            stats["q2_1m"] += 1
            stats["explicable"] += 1
            continue

        # Q3
        expm1, stm1 = solve_explanation_m1(omega, verbose=False)
        if stm1 == "OPTIMAL":
            stats["q3_m1"] += 1
            stats["explicable"] += 1
            continue

        # Q4 (try only if Q1-Q3 failed, to save time)
        trade1m, tradem1, st4 = solve_explanation_combined_q4(omega, feature_names=feature_names, verbose=False)
        if st4 == "OPTIMAL":
            stats["q4_combined"] += 1
            stats["explicable"] += 1

    return stats

print(" Calcul des statistiques d'explicabilité (solveur)...")
stats = calculer_statistiques(500)

print(f"\n{'='*60}")
print("STATISTIQUES D'EXPLICABILITÉ (solveur Q1-Q4)")
print(f"{'='*60}")
print(f"\nSur {stats['total']} comparaisons analysées :")
print("\n{:<30} {:>10} {:>10}".format("Type d'explication", "Nombre", "%"))
print("-" * 52)
print(f"{' Dominance (aucun compromis)':<30} {stats['dominance']:>10} {100*stats['dominance']/stats['total']:>9.1f}%")
print(f"{' Q1 : 1-1':<30} {stats['q1_11']:>10} {100*stats['q1_11']/stats['total']:>9.1f}%")
print(f"{' Q2 : 1-m':<30} {stats['q2_1m']:>10} {100*stats['q2_1m']/stats['total']:>9.1f}%")
print(f"{' Q3 : m-1':<30} {stats['q3_m1']:>10} {100*stats['q3_m1']/stats['total']:>9.1f}%")
print(f"{' Q4 : combinée':<30} {stats['q4_combined']:>10} {100*stats['q4_combined']/stats['total']:>9.1f}%")
print("-" * 52)
print(f"{' TOTAL EXPLICABLE':<30} {stats['explicable']:>10} {100*stats['explicable']/stats['total']:>9.1f}%")


 Calcul des statistiques d'explicabilité (solveur)...

STATISTIQUES D'EXPLICABILITÉ (solveur Q1-Q4)

Sur 500 comparaisons analysées :

Type d'explication                 Nombre          %
----------------------------------------------------
 Dominance (aucun compromis)          282      56.4%
 Q1 : 1-1                             163      32.6%
 Q2 : 1-m                              30       6.0%
 Q3 : m-1                              15       3.0%
 Q4 : combinée                          5       1.0%
----------------------------------------------------
 TOTAL EXPLICABLE                     495      99.0%
